In [1]:
### Parameters
#fold=1
#dataset="monks-1"
#d = 4# depth
#D= d

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 18 22:11:32 2023

@author: khaniyev
"""

#%%

import gurobipy as gp
import gurobipy
import pandas as pd
import numpy as np
import time





In [3]:
import numpy as np
from typing import List, Tuple

def get_split_values(leaf_type: int, depth: int) -> np.ndarray:
    """
    Compute the binary split pattern for a given leaf_type over a tree of given depth.
    Returns an array of 0.0/1.0 of length `depth`, where each entry is 1.0−bit of leaf_type.
    """
    bits = np.empty(depth, dtype=float)
    rem_val = leaf_type
    for l in range(depth):
        power = 2 ** (depth - l - 1)
        bits[l] = rem_val // power
        rem_val -= int(bits[l] * power)
    # invert so that 0→1.0 and 1→0.0 matches the Cython version
    return 1.0 - bits


def get_misclassification_bag(
    x: np.ndarray,
    y_k: np.ndarray,
    leaf_pattern: Tuple[List[int], List[int]],
    leaf_type: int
) -> Tuple[float, int, np.ndarray]:
    """
    Mask off the rows of x/y_k according to the leaf_pattern and leaf_type,
    then compute the misclassification count and sample size.

    Parameters
    ----------
    x : array of shape (N, p)
        Binary feature matrix (values 0 or 1).
    y_k : array of shape (N, C)
        One-hot (or count) matrix of class labels.
    leaf_pattern : (pattern0, pattern1)
        A tuple of two lists of feature-indices: 
        - pattern0 = features to test when split_value==0
        - pattern1 = features to test when split_value==1
    leaf_type : int
        An integer in [0, 2**depth − 1] indicating the leaf.

    Returns
    -------
    misclassification : float
        Number of misclassified samples = num_samples − max_class_count.
    num_samples : int
        Total number of samples reaching this leaf.
    mask : ndarray of bool, shape (N,)
        Boolean mask indicating which rows were selected.
    """
    # total depth is sum of lengths of both patterns
    depth = len(leaf_pattern[0]) + len(leaf_pattern[1])
    split_values = get_split_values(leaf_type, depth)

    # start with all True
    mask = np.ones(x.shape[0], dtype=bool)

    # for each unique split (0.0 or 1.0), refine mask by enforcing all features in the corresponding pattern
    for split in set(split_values):
        split_int = int(split)
        for feat in leaf_pattern[1 - split_int]:
            mask &= (x[:, feat] == split)

    # filter
    y_filt = y_k[mask]
    num_samples = y_filt.shape[0]

    if num_samples > 0:
        # count per class, take the largest
        class_counts = np.sum(y_filt, axis=0)
        max_class_sum = np.max(class_counts)
        misclassification = float(num_samples - max_class_sum)
    else:
        misclassification = 0.0

    return misclassification, num_samples, mask


loop

In [4]:

relpath="./Datasets/"+dataset+"/fold="+str(fold)+"_train.csv"
data = pd.read_csv(relpath)#pd.read_csv("./Datasets/fold=1_train.csv")

X = data.iloc[:,1:]
y = data.iloc[:,:1]
PP = [i for i in range(data.shape[1]-1)]
P = len(PP)
KK = list(data.y.unique())
K = len(KK)
D = d # depth
DD = list(range(D))
L = 2**D
LL = list(range(L))
NN = list(range(X.shape[0]))
N = len(NN)
y_k = np.zeros((X.shape[0], K))
for k in range(K):
    y_k[np.where(y==KK[k])[0],k] = 1

y_k= y_k.astype(np.int32)

x = np.array(X,dtype=np.int32)



relpath="./Datasets/"+dataset+"/fold="+str(fold)+"_test.csv"
data_test = pd.read_csv(relpath)#pd.read_csv("./Datasets/fold=1_train.csv")

X_test = data_test.iloc[:,1:]
y_test = data_test.iloc[:,:1]

y_k_test = np.zeros((X_test.shape[0], K))
for k in range(K):
    y_k_test[np.where(y_test==KK[k])[0],k] = 1

y_k_test=y_k_test.astype(np.int32)
x_test = np.array(X_test,dtype=np.int32)





relpath="./Datasets/"+dataset+"/fold="+str(fold)



In [5]:
def get_patterns(model,D,P):
    patterns_all=[]

    for leaf in range(2**D):
        node_splits=np.where(model.X[:(2**D-1)*P])  # should return 2**D-1 many
        node_splits_features_features=np.array(node_splits).flatten()//(2**D-1)
        node_splits_features_order=np.array(node_splits).flatten()%(2**D-1)

        # Get the indices that would sort the order_array
        sorted_indices = np.argsort(node_splits_features_order)
        node_splits_features=node_splits_features_features[sorted_indices]



        split_values=1-get_split_values(leaf,d).astype(int)
        w_indices_for_leaf=[0]

        ### obtain the leaf's association with split nodes
        for d_temp in range(1,d):
            values=np.arange(2**d_temp -1,2**d_temp -1 + 2**d_temp)
            w_index=leaf//(2**(d-d_temp))
            w_indices_for_leaf.append(values[w_index])


        pattern=[[],[]]

        for d_ in range(D):
            value=split_values[d_]
            w_index=w_indices_for_leaf[d_]
            pattern[value].append(node_splits_features[w_index])
        
        patterns_all.append(tuple(pattern))

    return patterns_all

def get_tree_mis(inc_patterns,d,x,y_k):
    miss=0
    for leaf in range(2**d):
        miss+=get_misclassification_bag(x, y_k, inc_patterns[leaf], leaf)[0]
    return miss/len(x)
    
    

In [6]:


if d==2:
    #%% Formulation 2 -- Where the pricing problems are flow-based

    oct_dw_4 = gp.Model()
    oct_dw_4.setParam("OutputFlag",1)
    oct_dw_4.setParam('TimeLimit', 3600)
    z = oct_dw_4.addVars(P, 3, vtype=gp.GRB.BINARY,name='node')
    a = oct_dw_4.addVars(P, L, vtype=gp.GRB.BINARY)
    b = oct_dw_4.addVars(P, L, vtype=gp.GRB.BINARY)
    t = oct_dw_4.addVars(N, L, vtype=gp.GRB.CONTINUOUS, ub=1)
    c = oct_dw_4.addVars(K, L, vtype=gp.GRB.BINARY)
    misclassification_error = gp.LinExpr((1/N)*gp.quicksum(t[i,l] for i in range(N) for l in range(L)))
    oct_dw_4.setObjective(misclassification_error, gp.GRB.MINIMIZE)
    oct_dw_4.addConstrs(gp.quicksum(z[j,branch] for j in range(P)) == 1 for branch in range(3)) 
    l=0
    oct_dw_4.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_4.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2)
    oct_dw_4.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 0)
    oct_dw_4.addConstrs(z[j,0] + z[j,1] == a[j,l] for j in range(P))
    l=1
    oct_dw_4.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_4.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1 )
    oct_dw_4.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_4.addConstrs(z[j,0] == a[j,l] for j in range(P))
    oct_dw_4.addConstrs(z[j,1] == b[j,l] for j in range(P))
    l=2
    oct_dw_4.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_4.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1 )
    oct_dw_4.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_4.addConstrs(z[j,0] == b[j,l] for j in range(P))
    oct_dw_4.addConstrs(z[j,2] == a[j,l] for j in range(P))
    l=3
    oct_dw_4.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_4.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 0 )
    oct_dw_4.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_4.addConstrs(z[j,0] + z[j,2]  == b[j,l] for j in range(P))
    oct_dw_4.addConstrs(gp.quicksum(c[k,l] for k in range(K)) == 1 for l in range(L)) 
    oct_dw_4.update()
    start_time=time.time()
    # oct_dw_lp  = oct_dw.relax()
    opt_sol = oct_dw_4.optimize()
    end_time=time.time()
    # TEST DATASET
    inc_patterns=get_patterns(oct_dw_4,D,P)
    miss_test=get_tree_mis(inc_patterns,d,x_test,y_k_test)

    # combine res
    res=np.array([1-oct_dw_4.objval,1-miss_test,end_time-start_time,oct_dw_4.MIPGap])

    labels=np.array(['Training Accuracy','Test Accuracy','Time','Opt Gap'])
    resWlabel=np.vstack([labels,res]).T

    np.savetxt(relpath+'_resCompact_'+str(d)+'.txt',resWlabel,delimiter=',',fmt='%s')


elif d==3:
    #%% Depth = 3
    oct_dw_d3_2 = gp.Model()
    oct_dw_d3_2.setParam("OutputFlag",1)
    oct_dw_d3_2.setParam('TimeLimit', 3600)
    z = oct_dw_d3_2.addVars(P, 7, vtype=gp.GRB.BINARY)
    a = oct_dw_d3_2.addVars(P, L, vtype=gp.GRB.BINARY)
    b = oct_dw_d3_2.addVars(P, L, vtype=gp.GRB.BINARY)
    t = oct_dw_d3_2.addVars(N, L, vtype=gp.GRB.CONTINUOUS, ub=1)
    c = oct_dw_d3_2.addVars(K, L, vtype=gp.GRB.BINARY)
    misclassification_error = gp.LinExpr((1/N)*gp.quicksum(t[i,l] for i in range(N) for l in range(L)))
    oct_dw_d3_2.setObjective(misclassification_error, gp.GRB.MINIMIZE)
    oct_dw_d3_2.addConstrs(gp.quicksum(z[j,branch] for j in range(P)) == 1 for branch in range(7)) 
    l=0
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 0)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,1] + z[j,3] == a[j,l] for j in range(P))
    l=1
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,1] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,3] == b[j,l] for j in range(P))
    l=2
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,4]  == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,1] == b[j,l] for j in range(P))
    l=3
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,0] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,1] + z[j,4] == b[j,l] for j in range(P))
    l=4
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstrs(z[j,2] + z[j,5] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] == b[j,l] for j in range(P))
    l=5
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,2] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,5] == b[j,l] for j in range(P))
    l=6
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,6] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,2] == b[j,l] for j in range(P))
    l=7
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 0 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,2] + z[j,6] == b[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(gp.quicksum(c[k,l] for k in range(K)) == 1 for l in range(L)) 
    oct_dw_d3_2.update()

    start_time=time.time()
    opt_sol = oct_dw_d3_2.optimize()
    end_time=time.time()

    # TEST DATASET
    inc_patterns=get_patterns(oct_dw_d3_2,D,P)
    miss_test=get_tree_mis(inc_patterns,d,x_test,y_k_test)
 
    # combine res
    res=np.array([1-oct_dw_d3_2.objval,1-miss_test,end_time-start_time,oct_dw_d3_2.MIPGap])
    labels=np.array(['Training Accuracy','Test Accuracy','Time','Opt Gap'])
    resWlabel=np.vstack([labels,res]).T

    np.savetxt(relpath+'_resCompact_'+str(d)+'.txt',resWlabel,delimiter=',',fmt='%s')


elif d==4:
    #%% Depth = 3
    oct_dw_d3_2 = gp.Model()
    cons_start=time.time()
    oct_dw_d3_2.setParam("OutputFlag",1)
    oct_dw_d3_2.setParam('TimeLimit', 3600)
    z = oct_dw_d3_2.addVars(P, 2**d-1, vtype=gp.GRB.BINARY)
    a = oct_dw_d3_2.addVars(P, L, vtype=gp.GRB.BINARY)
    b = oct_dw_d3_2.addVars(P, L, vtype=gp.GRB.BINARY)
    t = oct_dw_d3_2.addVars(N, L, vtype=gp.GRB.CONTINUOUS, ub=1)
    c = oct_dw_d3_2.addVars(K, L, vtype=gp.GRB.BINARY)

    misclassification_error = gp.LinExpr((1/N)*gp.quicksum(t[i,l] for i in range(N) for l in range(L)))
    oct_dw_d3_2.setObjective(misclassification_error, gp.GRB.MINIMIZE)
    oct_dw_d3_2.addConstrs(gp.quicksum(z[j,branch] for j in range(P)) == 1 for branch in range(2**d-1)) 
    
    l=0
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 4)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 0)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,1] + z[j,3]+ z[j,7] == a[j,l] for j in range(P))

    l=1
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 3 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,1] + z[j,3] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,7] == b[j,l] for j in range(P))

    l=2
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,1] + z[j,8] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,3] == b[j,l] for j in range(P))

    l=3
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,1] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,3] + z[j,8]== b[j,l] for j in range(P))


    l=4
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,4] + z[j,9]  == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,1] == b[j,l] for j in range(P))

    l=5
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,4] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,1] + z[j,9] == b[j,l] for j in range(P))


    l=6
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,10]  == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,1] + z[j,4] == b[j,l] for j in range(P))

    l=7
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1 )
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstrs(z[j,0] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,1] +  z[j,4]+ z[j,10] == b[j,l] for j in range(P))


    l=8
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstrs(z[j,2] + z[j,5] + z[j,11] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] == b[j,l] for j in range(P))

    l=9
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,2] + z[j,5]  == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,11] == b[j,l] for j in range(P))

    l=10
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,2] + z[j,12] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,5] == b[j,l] for j in range(P))

    l=11
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstrs(z[j,2] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,5] + z[j,12] == b[j,l] for j in range(P))


    l=12
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 2)
    oct_dw_d3_2.addConstrs(z[j,6] + z[j,13] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,2] == b[j,l] for j in range(P))

    l=13
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstrs(z[j,6] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,2] + z[j,13] == b[j,l] for j in range(P))


    l=14
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 1)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 3)
    oct_dw_d3_2.addConstrs(z[j,14] == a[j,l] for j in range(P))
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,2] + z[j,6] == b[j,l] for j in range(P))

    l=15
    oct_dw_d3_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*a[j,l] for j in range(P)) + gp.quicksum((1-x[i,j])*b[j,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    oct_dw_d3_2.addConstr(gp.quicksum(a[j,l] for j in range(P)) == 0)
    oct_dw_d3_2.addConstr(gp.quicksum(b[j,l] for j in range(P)) == 4)
    oct_dw_d3_2.addConstrs(z[j,0] + z[j,2] + z[j,6] + z[j,14] == b[j,l] for j in range(P))


    oct_dw_d3_2.addConstrs(gp.quicksum(c[k,l] for k in range(K)) == 1 for l in range(L)) 

    oct_dw_d3_2.update()
    cons_end=time.time()
    start_time=time.time()
    opt_sol = oct_dw_d3_2.optimize()
    end_time=time.time()


    # TEST DATASET
    inc_patterns=get_patterns(oct_dw_d3_2,D,P)
    miss_test=get_tree_mis(inc_patterns,d,x_test,y_k_test)
    
    # combine res
    res=np.array([1-oct_dw_d3_2.objval,1-miss_test,end_time-start_time,oct_dw_d3_2.MIPGap])
    labels=np.array(['Training Accuracy','Test Accuracy','Time','Opt Gap'])
    resWlabel=np.vstack([labels,res]).T

    np.savetxt(relpath+'_resCompact_'+str(d)+'.txt',resWlabel,delimiter=',',fmt='%s')





Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-26
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 8513 rows, 8737 columns and 138147 nonzeros
Model fingerprint: 0x2f2ee7a7
Variable types: 8000 continuous, 737 integer (737 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 2e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 0.5860000
Presolve removed 1714 rows and 1734 columns
Presolve time: 0.09s
Presolved: 6799 rows, 7003 columns, 105039 nonzeros
Variable types: 6338 continuous, 665 integer (665 binary)

Root relaxation: objective 0.000000e+00, 737 iterations, 0.05 seconds (0.13 work units)

    Nodes    |    Current Node    |     Objective Bounds

In [7]:
import sys

orig_stdout = sys.stdout
f = open(relpath+'_DecisionRules_Compact_'+str(d)+'.txt', 'w')
sys.stdout = f

print(inc_patterns)

sys.stdout = orig_stdout
f.close()